In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from datetime import datetime

class IkeaSofaScraper:
    def __init__(self):
        # 파일명 정의 - 날짜에 따라 파일명이 자동으로 기록됨
        date_str = datetime.now().strftime("%Y%m%d")
        self.ft_name = f'./ikea_sofa_crawl_{date_str}.txt'
        self.fc_name = f'./ikea_sofa_crawl_{date_str}.csv'
        self.fx_name = f'./ikea_sofa_crawl_{date_str}.xlsx'

        # Chrome Driver 설정
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--start-maximized')
        self.driver = webdriver.Chrome(options=chrome_options)

        # 전체 데이터 저장 구조
        self.sofa_data = {
            "제품 ID": [],
            "제품 이름": [],
            "가격": [],
            "제품 링크": [],
            "이미지 링크": []
        }

    def scrape_sofa_list(self):
        base_url = "https://www.ikea.com/kr/ko/cat/sofas-armchairs-fu003/"
        page_number = 1
        total_products = 0
        max_products = 390

        while total_products < max_products:
            url = f"{base_url}?page={page_number}"
            self.driver.get(url)

            # 페이지 로드 확인
            try:
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.plp-mastercard"))
                )
            except TimeoutException:
                print(f"페이지 로딩에 실패했습니다 (페이지 {page_number}). 스킵합니다.")
                break

            # 페이지 HTML 소스 파싱
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 소파 리스트 가져오기
            sofa_items = soup.find_all('div', class_='plp-mastercard')

            if not sofa_items:
                # 더 이상 제품이 없으면 종료
                print("더 이상 수집할 제품이 없습니다.")
                break

            for item in sofa_items:
                if total_products >= max_products:
                    break

                # 제품 ID 수집
                try:
                    product_id = item['data-product-number']
                except (TypeError, KeyError):
                    product_id = "Not Readable"

                # 제품 이름 수집
                try:
                    product_name = item['data-product-name']
                except (TypeError, KeyError):
                    product_name = "Not Readable"

                # 가격 수집
                try:
                    price = item.find('span', class_='plp-price__integer').get_text(strip=True)
                    currency = item.find('span', class_='plp-price__currency').get_text(strip=True)
                    price = f"{currency}{price}"
                except AttributeError:
                    price = "Not Available"

                # 제품 링크 수집
                try:
                    product_link = item.find('a', class_='plp-product__image-link')['href']
                    product_link = f"https://www.ikea.com{product_link}"
                except (TypeError, KeyError):
                    product_link = "Not Readable"

                # 이미지 링크 수집
                try:
                    image_link = item.find('img', class_='plp-product__image')['src']
                except (TypeError, KeyError, AttributeError):
                    image_link = "Not Available"

                # 데이터 저장
                self.sofa_data["제품 ID"].append(product_id)
                self.sofa_data["제품 이름"].append(product_name)
                self.sofa_data["가격"].append(price)
                self.sofa_data["제품 링크"].append(product_link)
                self.sofa_data["이미지 링크"].append(image_link)

                total_products += 1
                print(f"제품 ID: {product_id}, 이름: {product_name}, 가격: {price}, 링크: {product_link}, 이미지 링크: {image_link}")

            # 다음 페이지로 이동
            page_number += 1
            time.sleep(2)  # 페이지 로드 시간을 충분히 줌

    def save_data(self):
        # pandas DataFrame으로 변환
        df = pd.DataFrame(self.sofa_data)

        # 파일로 저장
        df.to_csv(self.fc_name, encoding="utf-8-sig", index=False)
        df.to_excel(self.fx_name, index=False)

        with open(self.ft_name, 'w', encoding='UTF-8') as f:
            f.write(str(df.to_string(index=False)))

        print(f"\n총 {len(df)} 건의 데이터 수집 완료")
        print("파일 저장 완료: txt 파일명:", self.ft_name)
        print("파일 저장 완료: csv 파일명:", self.fc_name)
        print("파일 저장 완료: xlsx 파일명:", self.fx_name)

    def close_driver(self):
        self.driver.quit()

# 실행 코드
if __name__ == "__main__":
    scraper = IkeaSofaScraper()
    scraper.scrape_sofa_list()
    scraper.save_data()
    scraper.close_driver()


In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import os
import time

class IkeaSofaImageScraper:
    def __init__(self):
        # 폴더명 정의 - 날짜에 따라 폴더명이 자동으로 기록됨
        date_str = time.strftime("%Y%m%d")
        self.image_folder = f'./ikea_sofa_images_{date_str}'

        # 이미지 저장 폴더 생성
        if not os.path.exists(self.image_folder):
            os.makedirs(self.image_folder)

        # Chrome Driver 설정
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--start-maximized')
        self.driver = webdriver.Chrome(options=chrome_options)

    def scrape_sofa_images(self):
        base_url = "https://www.ikea.com/kr/ko/cat/sofas-armchairs-fu003/"
        page_number = 1
        total_images = 0
        max_images = 390

        while total_images < max_images:
            url = f"{base_url}?page={page_number}"
            self.driver.get(url)

            # 페이지 로드 확인
            try:
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.plp-mastercard"))
                )
            except TimeoutException:
                print(f"페이지 로딩에 실패했습니다 (페이지 {page_number}). 스킵합니다.")
                break

            # 페이지 HTML 소스 파싱
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 소파 리스트 가져오기
            sofa_items = soup.find_all('div', class_='plp-mastercard')

            if not sofa_items:
                # 더 이상 제품이 없으면 종료
                print("더 이상 수집할 제품이 없습니다.")
                break

            for item in sofa_items:
                if total_images >= max_images:
                    break

                # 이미지 링크 수집
                try:
                    image_link = item.find('img', class_='plp-product__image')['src']
                except (TypeError, KeyError, AttributeError):
                    image_link = None

                if image_link:
                    self.download_image(image_link, total_images)
                    total_images += 1

            # 다음 페이지로 이동
            page_number += 1
            time.sleep(2)  # 페이지 로드 시간을 충분히 줌

    def download_image(self, url, count):
        try:
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                # 파일 경로 설정
                image_path = os.path.join(self.image_folder, f'sofa_image_{count + 1}.jpg')
                with open(image_path, 'wb') as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                print(f"이미지 저장 완료: {image_path}")
            else:
                print(f"이미지 다운로드 실패: {url}")
        except requests.RequestException as e:
            print(f"이미지 다운로드 중 에러 발생: {e}")

    def close_driver(self):
        self.driver.quit()

# 실행 코드
if __name__ == "__main__":
    scraper = IkeaSofaImageScraper()
    scraper.scrape_sofa_images()
    scraper.close_driver()

이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_1.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_2.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_3.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_4.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_5.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_6.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_7.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_8.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_9.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_10.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_11.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_12.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_13.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_14.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_15.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_16.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/sofa_image_17.jpg
이미지 저장 완료: ./ikea_sofa_images_20241106/s